# 1. Import libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import ETFs as etfs

Total number of ETFs: 33


# 2. Read assets from csv file

In [ ]:
data = pd.read_csv('assets_wide_minus1.csv', parse_dates=['Date'])
data.set_index('Date', inplace=True)
# display(data)
# 

,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,-1.000000,-1.000,-1.000000,42.619999,26.042215,-1.000000,-1.000000,12.307325,20.625570,14.720884,...,28.830000,100.450691,83.888580,317.519989,69.905640,-1.000000,27.664728,34.591946,33.707809,6.78375
2012-01-04,-1.000000,-1.000,-1.000000,42.509998,26.165094,-1.000000,-1.000000,12.452251,20.587933,14.581523,...,28.389999,100.608269,82.891190,318.160004,70.114922,-1.000000,27.328690,34.000534,33.888023,6.69750
2012-01-05,-1.000000,-1.000,-1.000000,41.509998,25.814005,-1.000000,-1.000000,12.290606,20.459963,14.412823,...,28.510000,100.876091,82.743690,313.359985,69.817894,-1.000000,26.831102,34.319885,33.472141,6.68250
2012-01-06,-1.000000,-1.000,-1.000000,41.349998,25.919333,-1.000000,-1.000000,12.128959,20.249193,14.148767,...,27.910000,100.616142,83.396942,313.760010,69.392540,-1.000000,26.462748,34.207523,33.070107,6.72625
2012-01-09,-1.000000,-1.000,-1.000000,42.189999,25.831558,-1.000000,-1.000000,12.167975,20.309414,14.207449,...,28.129999,100.860367,83.249443,312.640015,69.655838,-1.000000,26.663084,34.089241,33.097832,6.69625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,38.276001,10.405,100.550003,18.930000,14.140000,107.930000,56.490002,23.700001,41.509998,38.029999,...,29.959999,574.080017,88.790054,74.879997,129.030991,140.270004,71.730003,90.728004,74.690002,8.28750
2025-03-25,38.380001,10.440,100.720001,18.770000,14.030000,107.330002,56.580002,23.809999,41.709999,38.380001,...,30.580000,575.460022,88.780167,74.820000,129.460007,140.479996,72.160004,89.709999,75.089996,8.14875
2025-03-26,37.854000,10.400,98.169998,18.559999,14.110000,105.839996,56.080002,23.709999,41.380001,37.560001,...,30.530001,568.590027,88.196602,75.419998,130.250000,139.595001,71.139999,90.059998,74.410004,8.14500


# 7. Create rolling windows

In [ ]:
# def create_X_y(data, window_size):
#     X, y = [], []
#     for i in range(window_size, len(data) - 1):
#         window = data.iloc[i-window_size:i]
#         target = data.iloc[i+1]
#         if (window.values < 0).any() or (target.values < 0).any():
#             continue
#         X.append(window.values.flatten()) 
#         y.append(target.values) 
#     return np.array(X), np.array(y)

def create_X_y_rolling(df, window_size):
    X, y, first_values = [], [], []
    for i in range(len(df) - window_size):
        window = df.iloc[i:i+window_size]
        first_value = window.iloc[0]
        normalized_window = window / first_value  
        X.append(normalized_window.values.flatten())
        y_target = df.iloc[i+window_size] / first_value  
        y.append(y_target.values)
        first_values.append(first_value.values) 
    X = np.array(X)
    y = np.array(y)
    first_values = np.array(first_values)
    return X, y, first_values


window_size = 30
X, y, y_shift, first_values = create_X_y_rolling(data, window_size)
window_size = 30
X, y, first_values = create_X_y_rolling(data, window_size)
print(X.shape)
print(y.shape)
display(X)

(137, 990)
(137, 33)


array([[1.91432498, 2.27728239, 2.96800003, ..., 2.68511837, 2.13386644,
        1.22756591],
       [1.89715762, 2.26077475, 2.89249992, ..., 2.68637641, 2.15423718,
        1.22627607],
       [1.90173562, 2.26715301, 2.94799995, ..., 2.72867361, 2.15397236,
        1.22830299],
       ...,
       [1.75083844, 1.72963251, 2.40475006, ..., 2.72747269, 2.14811448,
        1.07278424],
       [1.74061462, 1.65809163, 2.40699997, ..., 2.76544879, 2.15797245,
        1.05951722],
       [1.75038178, 1.62133288, 2.42662506, ..., 2.79435593, 2.14753483,
        1.06006999]])

In [ ]:
dataset_size = len(X)
print(f"dataset_size: {dataset_size}")
train_size = int(dataset_size * 0.8)
test_size = int(dataset_size * 0.1)
val_size = dataset_size - train_size  - test_size

print(f"train_size: {train_size}, val_size: {val_size}, test_size: {test_size}")
print(f"dataset_size: {dataset_size}, window_size: {window_size}")

X_train, X_val, X_test = X[:train_size], X[train_size: train_size + val_size], X[train_size + val_size: ]
y_train, y_val, y_test= y[:train_size], y[train_size: train_size + val_size], y[train_size + val_size: ]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape
first_values_test = first_values[train_size + val_size:]


dataset_size: 137
train_size: 109, val_size: -2, test_size: 30
dataset_size: 137, window_size: 30


((109, 990), (109, 33), (0, 990), (0, 33), (30, 990), (30, 33))

In [6]:
rf = MultiOutputRegressor(
    RandomForestRegressor(
        random_state=42,
        criterion='squared_error',
        n_jobs=-1,
    )
    )
rf.fit(X_train, y_train)


MultiOutputRegressor(estimator=RandomForestRegressor(n_jobs=-1,
                                                     random_state=42))

In [ ]:
# index_series = data.index[train_size + val_size + window_size:]

Y_pred = rf.predict(X_test)
y_pred_price = Y_pred * first_values_test
y_pred_df = pd.DataFrame(y_pred_price, index=data.columns)
display(y_pred_df.T)

,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
0,38.083743,10.268489,97.650395,17.878187,13.978005,123.223936,59.321837,25.49045,40.378511,32.581859,...,29.016383,569.982822,91.688681,70.569157,116.548554,138.295102,67.76315,94.206888,74.256629,7.397713


In [ ]:
predicted_prices = pd.DataFrame(Y_pred * first_values_test, index=data.index[-len(Y_pred):], columns=data.columns)
predicted_prices.index = data.index[-len(predicted_prices):]
display(predicted_prices.T)

Date,2025-02-17,2025-02-18,2025-02-19,2025-02-20,2025-02-21,2025-02-24,2025-02-25,2025-02-26,2025-02-27,2025-02-28,...,2025-03-17,2025-03-18,2025-03-19,2025-03-20,2025-03-21,2025-03-24,2025-03-25,2025-03-26,2025-03-27,2025-03-28
AIEQ,38.394901,38.165726,38.180990,38.145402,38.182611,38.165386,38.072141,38.006872,38.057881,38.128268,...,38.074361,38.229002,38.081440,38.104584,38.117014,38.129107,38.161588,38.181591,38.188186,38.249851
ASHR.L,10.538784,10.497103,10.545455,10.604966,10.557119,10.487025,10.517783,10.241648,10.090176,10.087339,...,10.111631,10.355812,10.427517,10.002712,10.374842,10.436125,10.504354,10.426403,10.407954,9.959400
BCHN.L,97.498603,97.497703,97.524253,97.374652,97.162602,96.966552,96.755902,96.839402,96.961502,96.826602,...,97.242502,97.774851,97.934601,98.349551,98.225601,98.748951,98.666251,99.114701,100.343851,100.351301
CORN,17.650300,17.675701,17.730900,17.727900,17.726200,17.710900,17.808700,17.818700,17.811300,17.858900,...,17.912800,17.904000,17.918100,17.966500,17.957600,17.996300,17.946400,17.956100,17.994000,18.043300
DBO,13.601369,13.754876,13.695859,13.968114,13.937649,13.925665,13.770208,13.987099,13.959752,14.002784,...,13.844851,14.088181,14.106426,14.145461,14.137800,14.141978,14.124896,14.116701,14.051222,14.016257
EDEN,122.509663,122.426591,123.254096,123.282905,123.353085,123.328404,123.150600,122.967916,122.859407,123.188524,...,123.253716,123.523670,123.109169,123.041922,122.928976,123.170120,123.228338,123.310580,123.148254,123.138083
EMXC,59.933468,59.725075,59.645506,59.556623,59.551636,59.348600,59.313077,59.360143,59.385318,59.392636,...,59.146063,59.056861,58.933704,59.038570,59.114930,59.227011,59.214001,59.096507,59.066218,59.045132
EWA,25.974184,25.809951,25.704329,25.715593,25.687079,25.635068,25.604287,25.624441,25.636031,25.542787,...,25.377678,25.325311,25.296395,25.351549,25.330367,25.297433,25.339852,25.344605,25.301454,25.278393
EWC,41.352829,41.202039,41.192270,41.072960,40.983323,40.943257,40.841711,40.840547,40.676205,40.763605,...,39.929730,39.738794,39.834368,39.982012,39.879034,39.794182,39.810184,39.774185,39.761655,39.860846
EWG,33.145771,32.902733,32.940044,32.952174,32.872441,32.856478,32.814869,32.763523,32.752852,32.702211,...,32.378623,32.298054,32.331224,32.300036,32.315322,32.235581,32.251368,32.251173,32.277634,32.275411


In [15]:
yesterday_prices = data.iloc[-2]  # second last row 

today_prices_actual = data.iloc[-1] # last row

today_prices_predicted = pd.Series(y_pred_price.flatten(), index=data.columns)


comparison_df = pd.DataFrame({
    'Yesterday_actual': yesterday_prices,
    'Today_actual': today_prices_actual,
    'Today_pred': today_prices_predicted,
})

tickers_names = data.columns

correct_list = []
for ticker in tickers_names:
    yesterday = yesterday_prices[ticker]
    actual_today = today_prices_actual[ticker]
    predicted = today_prices_predicted[ticker]

    sign_actual = np.sign(actual_today - yesterday)
    sign_pred = np.sign(predicted - yesterday)

    if sign_actual == sign_pred:
        correct_list.append(1)
    else:
        correct_list.append(0)
comparison_df['Correct'] = correct_list

count_of_ones = correct_list.count(1)
print(f"Count of correct predictions: {count_of_ones}")
print(f"accuracy: {count_of_ones / len(correct_list)}")
display(comparison_df)
save_df = comparison_df.copy()
save_df['Yesterday actual'] = save_df['Yesterday_actual'].apply(lambda x: round(x, 2))
save_df['Today actual'] = save_df['Today_actual'].apply(lambda x: round(x, 2))
save_df['Today prediction'] = save_df['Today_pred'].apply(lambda x: round(x, 2))
save_df['Correct'] = save_df['Correct'].apply(lambda x: round(x, 2))
save_df.to_csv('rf_regression.csv')


Count of correct predictions: 20
accuracy: 0.6060606060606061


,Yesterday_actual,Today_actual,Today_pred,Correct
AIEQ,37.658001,36.845001,38.083743,0
ASHR.L,10.460000,10.340000,10.268489,1
BCHN.L,97.080002,92.714996,97.650395,0
CORN,18.510000,18.459999,17.878187,1
DBO,14.130000,13.970000,13.978005,1
EDEN,105.330002,105.169998,123.223936,0
EMXC,56.110001,55.180000,59.321837,0
EWA,23.750000,23.469999,25.490450,0
EWC,41.250000,40.570000,40.378511,1
EWG,37.590000,37.389999,32.581859,1
